In [20]:
import pandas as pd

campus = pd.read_excel('campi/CAFI.xlsx')
campus['data_entrada'] = pd.to_datetime(campus['data_entrada'], utc=True)
campus = campus[campus['data_entrada'] < '2021-05-01']
# campus['valor'] = pd.to_numeric(campus['valor'])
# display(campus.info())
campus = campus[['numero', 'categoria_codigo', 'descricao']]
# display(campus)

priori = pd.read_excel('priori/AFI - Afogados.xlsx')
priori = priori[['N° Ativo', 'Nome do custo', 'Descrição']]
priori = priori.rename(columns={'N° Ativo': 'numero'})
# display(priori)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9617 entries, 0 to 9616
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   Unnamed: 0             9617 non-null   int64              
 1   id                     9617 non-null   int64              
 2   numero                 9617 non-null   int64              
 3   categoria_codigo       9617 non-null   object             
 4   categoria_nome         9617 non-null   object             
 5   conta_contabil_codigo  9617 non-null   int64              
 6   conta_contabil         9617 non-null   object             
 7   status                 9617 non-null   object             
 8   numeroserie            9 non-null      object             
 9   descricao              9617 non-null   object             
 10  sala                   8385 non-null   object             
 11  sala_predio            8385 non-null   object           

None

,numero,Nome do custo,Descrição
0,8878,IFPE449052-04 - APARELHOS DE MEDICAO E ORIENTACAO,37224CREC - BALANCA (MODELO L CAPACIDADE 2KG D...
1,9461,"IFPE449052-08 - EQUIPUtens-MED,OdontLabto,Hosp",39167CREC - CADEIRA DE RODAS (C/ASSENTO EM COU...
2,11620,IFPE449052-35 - EQUIP DE PROCESSAMENTO DE DADOS,44010CREC - MICROCOMPUTADOR (C/PROCES.INTEL PE...
3,12582,IFPE449052-35 - EQUIP DE PROCESSAMENTO DE DADOS,45795CREC - MICROCOMPUTADOR (CPU PENTIUM 4 2.4...
4,12586,IFPE449052-35 - EQUIP DE PROCESSAMENTO DE DADOS,45799CREC - MICROCOMPUTADOR (CPU PENTIUM 4 2.4...
...,...,...,...
8348,9160702,IFPE449052-18 - COLECOES E MATERIAS BIBLIOGRAF...,TOPOGRAFIA APLICADA A ENGENHARIA - VOL 2 - BOR...
8349,9160802,IFPE449052-18 - COLECOES E MATERIAS BIBLIOGRAF...,TOPOGRAFIA APLICADA A ENGENHARIA - VOL 2 - BOR...
8350,9160902,IFPE449052-18 - COLECOES E MATERIAS BIBLIOGRAF...,TOPOGRAFIA APLICADA A ENGENHARIA - VOL 2 - BOR...
8351,9161002,IFPE449052-18 - COLECOES E MATERIAS BIBLIOGRAF...,TOPOGRAFIA APLICADA A ENGENHARIA - VOL 2 - BOR...


In [3]:
# compara = pd.merge(cafi, nan, on='NUMERO', how='outer', indicator=True)
# compara.to_excel('campi/compara.xlsx')